train, validation은 일단 나눔. 그리고 cbam적용하는 것이 목표임.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torchaudio
!pip install skorch

In [3]:
import pandas as pd
from pathlib import Path
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from torch.utils.data import Subset

In [4]:
Catholic_file = '/content/drive/MyDrive/breath_data/breath_v2.2/aug_train_v2.2.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class,sex,old
0,0002-1.wav,non-wheezing,0,1,7
1,0002-2.wav,non-wheezing,0,1,7
2,0002-3.wav,non-wheezing,0,1,7
3,0002-4.wav,non-wheezing,0,1,7
4,0003-2.wav,non-wheezing,0,0,1
...,...,...,...,...,...
1605,wn_0535-4.wav,wheezing,1,0,9
1606,wn_0588-1.wav,wheezing,1,0,1
1607,wn_0599-3.wav,wheezing,1,1,6
1608,wn_0602-2.wav,wheezing,1,1,8


In [5]:
df = df.sample(frac=1)

In [6]:
df

,filename,category,class,sex,old
1233,ts_0032-3.wav,non-wheezing,0,1,2
1243,ts_0035-4.wav,non-wheezing,0,1,3
458,cp_0602-2.wav,wheezing,1,1,8
1116,rs_0597-4.wav,wheezing,1,0,1
79,0031-2.wav,non-wheezing,0,0,1
...,...,...,...,...,...
1096,rs_0559-1.wav,wheezing,1,0,11
391,cp_0513-2.wav,wheezing,1,0,7
1095,rs_0550-4.wav,wheezing,1,0,5
1056,rs_0535-2.wav,non-wheezing,0,0,9


In [7]:
mean = df.old.mean()
std = df.old.std()
df.old = (df.old - mean) / std

In [8]:
df

,filename,category,class,sex,old
1233,ts_0032-3.wav,non-wheezing,0,1,-0.639293
1243,ts_0035-4.wav,non-wheezing,0,1,-0.330391
458,cp_0602-2.wav,wheezing,1,1,1.214119
1116,rs_0597-4.wav,wheezing,1,0,-0.948195
79,0031-2.wav,non-wheezing,0,0,-0.948195
...,...,...,...,...,...
1096,rs_0559-1.wav,wheezing,1,0,2.140826
391,cp_0513-2.wav,wheezing,1,0,0.905217
1095,rs_0550-4.wav,wheezing,1,0,0.287413
1056,rs_0535-2.wav,non-wheezing,0,0,1.523021


In [9]:
df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class', 'sex', 'old']]
df.head()

,relative_path,class,sex,old
1233,/ts_0032-3.wav,0,1,-0.639293
1243,/ts_0035-4.wav,0,1,-0.330391
458,/cp_0602-2.wav,1,1,1.214119
1116,/rs_0597-4.wav,1,0,-0.948195
79,/0031-2.wav,0,0,-0.948195


In [10]:
data_path = '/content/drive/MyDrive/breath_data/breath_v2.2/aug_train_v2.2'

In [11]:
class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)

In [12]:
class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #여기부터 나이, 성별변수를 집어넣는 과정임
    x = self.df.loc[idx, 'old']
    y = self.df.loc[idx, 'sex']
    x = torch.from_numpy(np.asarray(x).reshape((1,)))
    y = torch.from_numpy(np.asarray(y).reshape((1,)))
    tabular = torch.cat((x, y), 0)
    tabular = tabular.float()
    #print(tabular)
    #x1 = ["sex", "old"]
    #x2 = x2.
    #x2 = x2.iloc[idx].values
    
    
    return aug_sgram, class_id

In [13]:
brds = breathDS(df, data_path)

In [14]:
num_items = len(brds)
print(len(brds))
num_train = round(num_items * 0.9)
num_val = num_items - num_train
train_ds, val_ds = random_split(brds, [num_train, num_val])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=True)

1610


In [15]:
#train_dl = DataLoader(brds, batch_size=16, shuffle=True)

In [16]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [17]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


In [18]:
Model1 = ResNet(BasicBlock, [2, 2, 2, 2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device

device(type='cuda', index=0)

In [19]:
def training(model, train_dl, val_dl, num_epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0
    correct_prediction_1 = 0
    total_prediction_1 = 0

    for i, data in enumerate(train_dl):
        x, labels = data[0].to(device), data[1].to(device)
        #inputs, inputs2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        #여기에도 inputs2를 추가시켜줫음. 이제 data[1]이 가리키는건 breathDS의 tablular임
        x_m, x_s = x.mean(), x.std()
        x = (x - x_m) / x_s
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
        _, prediction = torch.max(outputs,1)
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

    for i, data in enumerate(val_dl):
        x, labels = data[0].to(device), data[1].to(device)
        #inputs, inputs2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        #여기에도 inputs2를 추가시켜줫음. 이제 data[1]이 가리키는건 breathDS의 tablular임
        x_m, x_s = x.mean(), x.std()
        x = (x - x_m) / x_s
        outputs = model(x)
        _, prediction = torch.max(outputs,1)
        correct_prediction_1 += (prediction == labels).sum().item()
        total_prediction_1 += prediction.shape[0]

    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    val_acc = correct_prediction_1/total_prediction_1
    print(f'Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}, Validation Accuracy: {val_acc:.4f}')

  print('Finished Training')

In [20]:
num_epochs=30
training(Model1, train_dl, val_dl, num_epochs)

Epoch: 0, Loss: 0.4135, Accuracy: 0.8316, Validation Accuracy: 0.8447
Epoch: 1, Loss: 0.3935, Accuracy: 0.8544, Validation Accuracy: 0.8199
Epoch: 2, Loss: 0.3913, Accuracy: 0.8585, Validation Accuracy: 0.7950
Epoch: 3, Loss: 0.3711, Accuracy: 0.8558, Validation Accuracy: 0.8571
Epoch: 4, Loss: 0.3572, Accuracy: 0.8606, Validation Accuracy: 0.8634
Epoch: 5, Loss: 0.3696, Accuracy: 0.8592, Validation Accuracy: 0.8447
Epoch: 6, Loss: 0.3665, Accuracy: 0.8509, Validation Accuracy: 0.8509
Epoch: 7, Loss: 0.3644, Accuracy: 0.8661, Validation Accuracy: 0.8447
Epoch: 8, Loss: 0.3599, Accuracy: 0.8578, Validation Accuracy: 0.8075
Epoch: 9, Loss: 0.3525, Accuracy: 0.8640, Validation Accuracy: 0.8447
Epoch: 10, Loss: 0.3490, Accuracy: 0.8551, Validation Accuracy: 0.8509
Epoch: 11, Loss: 0.3234, Accuracy: 0.8709, Validation Accuracy: 0.8137
Epoch: 12, Loss: 0.3386, Accuracy: 0.8634, Validation Accuracy: 0.8696
Epoch: 13, Loss: 0.3021, Accuracy: 0.8772, Validation Accuracy: 0.8634
Epoch: 14, Loss:

In [21]:
Catholic_file1 = '/content/drive/MyDrive/breath_data/breath_v2.2/test_v2.2.csv'
df1 = pd.read_csv(Catholic_file1)
df1

,filename,category,class,sex,old
0,0001-2.wav,non-wheezing,0,0,0
1,0003-1.wav,non-wheezing,0,0,1
2,0004-1.wav,non-wheezing,0,0,1
3,0004-4.wav,non-wheezing,0,0,1
4,0006-1.wav,non-wheezing,0,1,1
5,0007-1.wav,non-wheezing,0,0,7
6,0008-1.wav,non-wheezing,0,0,2
7,0009-1.wav,non-wheezing,0,0,3
8,0010-3.wav,non-wheezing,0,1,3
9,0012-1.wav,non-wheezing,0,0,2


In [22]:
mean = df1.old.mean()
std = df1.old.std()
df1.old = (df1.old - mean) / std

In [23]:
df1['relative_path'] = '/' + df1['filename'].astype(str)
df1 = df1[['relative_path', 'class', 'sex', 'old']]
df1.head()

,relative_path,class,sex,old
0,/0001-2.wav,0,0,-1.131772
1,/0003-1.wav,0,0,-0.809217
2,/0004-1.wav,0,0,-0.809217
3,/0004-4.wav,0,0,-0.809217
4,/0006-1.wav,0,1,-0.809217


In [24]:
data_path = '/content/drive/MyDrive/breath_data/breath_v2.2/test_v2.2'

In [25]:

class Breath_sound_Util1():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)

  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

In [26]:
class breathDS1(Dataset):
    
  def __init__(self, df1, data_path):
    self.df1 = df1
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
            
  def __len__(self):
    return len(self.df1)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df1.loc[idx, 'relative_path']
    class_id = self.df1.loc[idx, 'class']
    
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #여기부터 나이, 성별변수를 집어넣는 과정임
    x = self.df1.loc[idx, 'old']
    y = self.df1.loc[idx, 'sex']
    x = torch.from_numpy(np.asarray(x).reshape((1,)))
    y = torch.from_numpy(np.asarray(y).reshape((1,)))
    tabular = torch.cat((x, y), 0)
    tabular = tabular.float()
    #print(tabular)
    #x1 = ["sex", "old"]
    #x2 = x2.
    #x2 = x2.iloc[idx].values
    
    
    return aug_sgram, class_id

In [27]:
brds1 = breathDS1(df1, data_path)

In [28]:
test_dl = torch.utils.data.DataLoader(brds1, batch_size=16, shuffle=True)

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score

def inference (model, test_dl):
  correct_prediction = 0
  total_prediction = 0
  TP = 0 
  TN = 0
  FN = 0
  FP = 0
  a = []
  b = []
  with torch.no_grad():
    for i, data in enumerate(test_dl):
      
      x, labels = data[0].to(device), data[1].to(device)
      cpu_labels = data[1]
      x_m, x_s = x.mean(), x.std()
      x = (x - x_m) / x_s
      outputs = model(x)
 
      _, prediction = torch.max(outputs,1)
      
      cpu_prediction = prediction.to('cpu')
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
          

      TP += ((prediction == 1) & (labels == 1)).sum()
# TN predict and label are both 0
      TN += ((prediction == 0) & (labels == 0)).sum()
# FN    predict 0 label 1
      FN += ((prediction == 0) & (labels == 1)).sum()
# FP    predict 1 label 0
      FP += ((prediction == 1) & (labels == 0)).sum()

      c = cpu_labels.tolist()
      a.extend(c)

      d = cpu_prediction.tolist()
      b.extend(d)
    
    # TP : 휘징인데 휘징이라고 판단(잘함)
    # FP : 헬시인데 휘징이라고 판단(못함)
    # TN : 헬시인데 헬시라고 판단(잘함)
    # FN : 휘징인데 헬시라고 판단(못함)
    
    # POSITIVE : 1(휘징)이라고 판단
    # NEGATIVE : 0(헬시)라고 판단
    # TRUE : 정답값 1(휘징)
    # FALSE : 정답값 0(헬시)
    
    # PRECISION : 휘징이라고 판단한 것 중 진짜 휘징
    # RECALL : 휘징인데 휘징이라고 잘 판단
    

  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  F1 = 2 * recall * precision / (recall + precision)
  print(TP)
  print(FP)
  print(FN)
  print(TN)

  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.4f}, Total items: {total_prediction}')
  print(f'precision: {precision:.4f}, F1: {F1:.4f}')
  print(f'recall: {recall:.4f}')
  target_names = ['healthy', 'wheezing']
  print(classification_report(a, b, target_names = target_names))
  print("AUC:{}".format(roc_auc_score(a, b)))

inference(Model1, test_dl)

tensor(19, device='cuda:0')
tensor(3, device='cuda:0')
tensor(2, device='cuda:0')
tensor(33, device='cuda:0')
Accuracy: 0.9123, Total items: 57
precision: 0.8636, F1: 0.8837
recall: 0.9048
              precision    recall  f1-score   support

     healthy       0.94      0.92      0.93        36
    wheezing       0.86      0.90      0.88        21

    accuracy                           0.91        57
   macro avg       0.90      0.91      0.91        57
weighted avg       0.91      0.91      0.91        57

AUC:0.9107142857142856
